In [1]:
import numpy as np
import pandas as pd
import math
import random
import scipy as sp
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import json
from pathlib import Path
from skimage.io import imread, imshow, concatenate_images
from skimage import io, transform
from skimage.measure import label, regionprops
from sklearn.model_selection import train_test_split
from PIL import Image 
import os
import sys
import cv2
import gc

In [2]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    https://www.kaggle.com/paulorzp/run-length-encode-and-decode
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def multi_rle_encode(img):
    labels = label(img)
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]
        
def maskcsv_to_img(masks, img_name,shape=(768,768)):
    masks_img = np.zeros((768,768))
    boolean=masks['ImageId']==img_name
    bins=masks.loc[boolean,'EncodedPixels']
    bins=bins.tolist()
    
    for m in bins:
        if str(m)==m:
            masks_img=masks_img+rle_decode(m)
            
    masks_img=np.expand_dims(masks_img, -1)   
    
    return masks_img

def masks_as_image(groupix,shape=(768, 768)):
    masks = np.zeros(shape, dtype = np.int)
    
    for m in groupix:
        if str(m)==m:
            masks += rle_decode(m)
            
    masks=np.expand_dims(masks, -1)
    
    return masks

def imshow_mask(img, mask):
    img = img.numpy()
    img = img.transpose((1, 2, 0))
    img = np.clip(img, 0, 1)
    
    mask = mask.numpy()
    mask = mask.transpose((1, 2, 0))
    mask = np.clip(mask, 0, 1)
    
    fig, axs = plt.subplots(1,2, figsize=(15,30))
    
    axs[0].imshow(img)
    axs[1].imshow(mask)
    axis('off')

def clip_fin(gt):
    return np.clip(gt.numpy().transpose((1, 2, 0)), 0, 1)

def imshow_gt_out(img, mask, mask_out):
    img = clip_fin(img)
    mask = clip_fin(mask)
    mask_out = clip_fin(mask_out)
    
    fig, ax = plt.subplots(1,3, figsize=(10,30))
    
    tab1=[img,mask,mask_out]
    tab2=['Original','Actual','Model']
    
    for i in range(3):
        ax[i].imshow(tab1[i])
        ax[i].set_title(tab2[i])
        
    axis('off')

def imshow_overlay(img, mask, title=None):
    """Imshow for Tensor."""
    img=clip_fin(img)
    mask=clip_fin(mask)
    
    fig = plt.figure()
    plt.imshow(mask_overlay(img, mask))

def mask_overlay(image, mask, color=(0, 1, 0)):
    mask = np.dstack((mask, mask, mask)) * np.array(color)
    weighted_sum = cv2.addWeighted(mask, 0.5, image, 0.5, 0.)
    img = image.copy()
    ind = mask[:, :, 1] > 0
    img[ind] = weighted_sum[ind]    
    return img

In [3]:
class Dataset(Dataset):
    
    def __init__(self, ids, transform=None, mode='train'):
        #Mode can take values : 
        # - train
        # - val : to evaluate perf
        # - test : for the submission
        
        IDs=list(ids.groupby('ImageId'))

        self.ids =  [ID for ID, _ in IDs] 
        
        self.masks = [mask['EncodedPixels'].values for _,mask in IDs]

        self.trans = transform
        
        self.img_transform = transforms.Compose([transforms.ToTensor()])
        
        self.mode=mode
        
               
    def __getitem__(self, idx):
        
        ID = self.ids[idx]
        
        if (self.mode == 'train'):
            path = os.path.join('../input/airbus-ship-detection/train_v2',ID)
        
        elif(self.mode == 'validation'):
            path = os.path.join('../input/airbus-ship-detection/train_v2',ID)
            
        else:
            path = os.path.join('../input/airbus-ship-detection/test_v2',ID)
            
        image = imread(path)
        
        mask = masks_as_image(self.masks[idx])
        
        if (self.trans): 
            image, mask = self.trans(image, mask)
            
        if (self.mode == 'train'):
            x=self.img_transform(image)
            y=torch.from_numpy(np.moveaxis(mask, -1, 0)).float()
            return x,y   
        
        elif (self.mode == 'validation'):
            x=self.img_transform(image)
            y=torch.from_numpy(np.moveaxis(mask, -1, 0)).float()
            return x,y    
            
        else:
            x=self.img_transform(image)
            y=str(ID)
            return x,y 

    def __len__(self):
        return len(self.ids)

In [4]:
# https://github.com/ternaus/robot-surgery-segmentation

def clip(img, dtype, maxval):
    return np.clip(img, 0, maxval).astype(dtype)

class DualCompose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, x, mask=None):
        for t in self.transforms:
            x, mask = t(x, mask)
        return x, mask

class ImageOnly:
    def __init__(self, trans):
        self.trans = trans

    def __call__(self, x, mask=None):
        return self.trans(x), mask


class VerticalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            img = cv2.flip(img, 0)
            if mask is not None:
                mask = cv2.flip(mask, 0)
        return img, mask


class HorizontalFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            img = cv2.flip(img, 1)
            if mask is not None:
                mask = cv2.flip(mask, 1)
        return img, mask


class RandomFlip:
    def __init__(self, prob=0.5):
        self.prob = prob

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            d = random.randint(-1, 1)
            img = cv2.flip(img, d)
            if mask is not None:
                mask = cv2.flip(mask, d)
        return img, mask


class Rotate:
    def __init__(self, limit=90, prob=0.5):
        self.prob = prob
        self.limit = limit

    def __call__(self, img, mask=None):
        if random.random() < self.prob:
            angle = random.uniform(-self.limit, self.limit)

            height, width = img.shape[0:2]
            mat = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1.0)
            img = cv2.warpAffine(img, mat, (height, width),
                                 flags=cv2.INTER_LINEAR,
                                 borderMode=cv2.BORDER_REFLECT_101)
            if mask is not None:
                mask = cv2.warpAffine(mask, mat, (height, width),
                                      flags=cv2.INTER_LINEAR,
                                      borderMode=cv2.BORDER_REFLECT_101)

        return img, mask

class RandomCrop:
    def __init__(self, size):
        self.h = size[0]
        self.w = size[1]

    def __call__(self, img, mask=None):
        height, width, _ = img.shape

        h_start = np.random.randint(0, height - self.h)
        w_start = np.random.randint(0, width - self.w)

        img = img[h_start: h_start + self.h, w_start: w_start + self.w,:]

        assert img.shape[0] == self.h
        assert img.shape[1] == self.w

        if mask is not None:
            if mask.ndim == 2:
                mask = np.expand_dims(mask, axis=2)
            mask = mask[h_start: h_start + self.h, w_start: w_start + self.w,:]

        return img, mask

class CenterCrop:
    def __init__(self, size):
        self.height = size[0]
        self.width = size[1]

    def __call__(self, img, mask=None):
        h, w, c = img.shape
        dy = (h - self.height) // 2
        dx = (w - self.width) // 2
        y1 = dy
        y2 = y1 + self.height
        x1 = dx
        x2 = x1 + self.width
        img = img[y1:y2, x1:x2,:]
        if mask is not None:
            if mask.ndim == 2:
                mask = np.expand_dims(mask, axis=2)
            mask = mask[y1:y2, x1:x2,:]

        return img, mask

In [5]:
# Data augmentation
train_transform = DualCompose([HorizontalFlip(), VerticalFlip(), RandomCrop((256,256,3))])
val_transform = DualCompose([CenterCrop((512,512,3))])

In [6]:
#https://www.kaggle.com/c/airbus-ship-detection/discussion/62921

traincorr=pd.read_csv('../input/corruption/corruptedtrainimages.csv')
removaltrain=traincorr['ImageId']

df = pd.read_csv('../input/airbus-ship-detection/train_ship_segmentations_v2.csv')
df = df[~df['ImageId'].isin(removaltrain)]

with_ships = df.dropna()
unique = with_ships.groupby('ImageId').size().reset_index(name='counts')

train_id, val_id = train_test_split(unique, test_size=0.05, stratify=unique['counts'])
train = pd.merge(with_ships, train_id)
validation = pd.merge(with_ships, val_id)

train_dataset = Dataset(train, transform=train_transform, mode='train')
validation_dataset = Dataset(validation, transform=val_transform, mode='validation')

In [7]:
def BCEDiceLoss(outputs, targets): 
    bce = nn.BCEWithLogitsLoss()
    loss = bce(outputs, targets)
    targets = (targets == 1.0).float().view(-1)
    outputs = F.sigmoid(outputs).view(-1)
    intersection = (outputs * targets).sum()
    dice = 2.0 * (intersection + 1)/(targets.sum()+outputs.sum()+1)
    loss -= torch.log(dice)
    return loss

In [8]:
class Down(torch.nn.Module):
    def __init__(self, input_channel, output_channel, down_size):
        super(Down, self).__init__()
        self.conv1 = torch.nn.Conv2d(input_channel, output_channel, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(output_channel)
        self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(output_channel)
        self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(output_channel)
        self.conv4 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn4 = torch.nn.BatchNorm2d(output_channel)
        self.max_pool = torch.nn.MaxPool2d(2, 2)
        self.relu = torch.nn.ReLU()
        self.down_size = down_size

    def forward(self, x):
        if self.down_size:
            x = self.max_pool(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        return x

class Up(torch.nn.Module):
    def __init__(self, prev_channel, input_channel, output_channel):
        super(Up, self).__init__()
        self.up_sampling = torch.nn.Upsample(scale_factor=2, mode='bilinear')
        self.conv1 = torch.nn.Conv2d(prev_channel + input_channel, output_channel, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(output_channel)
        self.conv2 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(output_channel)
        self.conv3 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn3 = torch.nn.BatchNorm2d(output_channel)
        self.conv4 = torch.nn.Conv2d(output_channel, output_channel, 3, padding=1)
        self.bn4 = torch.nn.BatchNorm2d(output_channel)
        self.relu = torch.nn.ReLU()

    def forward(self, prev_feature_map, x):
        x = self.up_sampling(x)
        x = torch.cat((x, prev_feature_map), dim=1)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.relu(self.bn2(self.conv2(x)))
        x = self.relu(self.bn3(self.conv3(x)))
        x = self.relu(self.bn4(self.conv4(x)))
        return x


class Final_UNet(torch.nn.Module):
    def __init__(self):
        super(Final_UNet, self).__init__()

        self.down_1 = Down(3, 8, False)
        self.down_2 = Down(8, 16, True)
        self.down_3 = Down(16, 32, True)
        self.down_4 = Down(32, 64, True)
        self.down_5 = Down(64, 128, True)
        self.down_6 = Down(128, 256, True)
        self.down_7 = Down(256, 512, True)
        self.down_8 = Down(512, 1024, True)

        self.relu = torch.nn.ReLU()
        self.mid_conv1 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
        self.bn1 = torch.nn.BatchNorm2d(1024)
        self.mid_conv2 = torch.nn.Conv2d(1024, 1024, 3, padding=1)
        self.bn2 = torch.nn.BatchNorm2d(1024)

        self.up_1 = Up(512, 1024, 512)
        self.up_2 = Up(256, 512, 256)
        self.up_3 = Up(128, 256, 128)
        self.up_4 = Up(64, 128, 64)
        self.up_5 = Up(32, 64, 32)
        self.up_6 = Up(16, 32, 16)
        self.up_7 = Up(8, 16, 8)

        self.last_conv = torch.nn.Conv2d(8, 1, 1, padding=0)

    def forward(self, x):
        self.x1 = self.down_1(x)
        self.x2 = self.down_2(self.x1)
        self.x3 = self.down_3(self.x2)
        self.x4 = self.down_4(self.x3)
        self.x5 = self.down_5(self.x4)
        self.x6 = self.down_6(self.x5)
        self.x7 = self.down_7(self.x6)
        self.x8 = self.down_8(self.x7)
        self.x8 = self.relu(self.bn1(self.mid_conv1(self.x8)))
        self.x8 = self.relu(self.bn2(self.mid_conv2(self.x8)))
        x = self.up_1(self.x7, self.x8)
        x = self.up_2(self.x6, x)
        x = self.up_3(self.x5, x)
        x = self.up_4(self.x4, x)
        x = self.up_5(self.x3, x)
        x = self.up_6(self.x2, x)
        x = self.up_7(self.x1, x)
        x = self.last_conv(x)
        return x

# PART 3 - CREATE TRAIN AND VALIDATION LOOPS

In [9]:
def train(lr, model, train_loader, valid_loader, n_epochs=8):
      
    device = torch.device("cuda")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    epoch = 1
    step = 0
    
    combloss=[]
    meanloss=[]

    for epoch in range(epoch, n_epochs + 1):
        model.train()
        tq = tqdm(total=len(train_loader) *  16)
        
        tq.set_description('Epoch ' + str(epoch))
        
        losses = []
        
        trainloader = train_loader
        
        for i, (inputs, targets) in enumerate(trainloader):
            #Put inputs and targets to cuda
            inputs = inputs.to("cuda")
            targets = targets.to("cuda")
            
            #Compute loss
            optimizer.zero_grad()
            outputs = model.forward(inputs)
            loss = BCEDiceLoss(outputs, targets)
            batch_size = inputs.size(0)
            loss.backward()
            optimizer.step()
            step += 1
            tq.update(batch_size)
            losses.append(loss.item())
            current_loss = np.mean(losses[-50:]) # we average on the last 50 
            tq.set_postfix(loss='{:.4f}'.format(current_loss))
            
            if i and i % 50 == 0:
                meanloss.append([step,current_loss])
                
                
        tq.close()

        # Validation
        combloss.append([step,validation(model, valid_loader)])
    
    return combloss,meanloss

In [10]:
def validation(model, valid_loader):
    
    losses = []
    device = torch.device("cuda")
    model.eval()
    
    for inputs, targets in valid_loader:
        inputs = inputs.to("cuda")
        targets = targets.to("cuda")
        outputs = model.forward(inputs)
        loss = BCEDiceLoss(outputs, targets)
        losses.append(loss.item())
    
    Loss = np.mean(losses)  
    
    return Loss

In [ ]:
# Training
model=Final_UNet()

combloss,meanloss=train(lr = 1e-4,model=model,
                        train_loader=torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True),
                        valid_loader=torch.utils.data.DataLoader(validation_dataset, batch_size=4, shuffle=True),
                        n_epochs = 8
                        )

Epoch 1:   0%|          | 0/39984 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:3063: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/opt/conda/lib/python3.7/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Epoch 3:  64%|██████▍   | 25616/39984 [12:44<07:00, 34.15it/s, loss=2.6480]

# PART 5 - SUBMISSION

In [ ]:
from skimage.morphology import binary_opening, disk

test = pd.DataFrame({'ImageId': os.listdir('../input/airbus-ship-detection/test_v2'), 'EncodedPixels': None})

loader = torch.utils.data.DataLoader(dataset=Dataset(test, mode='test'), batch_size=2)
    
out = []

for batch_num, (inputs, paths) in enumerate(tqdm(loader, desc='Test')):
    inputs = inputs.cuda()
    outputs = model(inputs)
    
    for i, image_name in enumerate(paths):
        mask = F.sigmoid(outputs[i,0]).data.detach().cpu().numpy()
        cur_seg = binary_opening(mask>0.5, disk(2))
        cur_rles = multi_rle_encode(cur_seg)
        if len(cur_rles)>0:
            for c_rle in cur_rles:
                out += [{'ImageId': image_name, 'EncodedPixels': c_rle}]
        else:
            out += [{'ImageId': image_name, 'EncodedPixels': None}]
        
submission = pd.DataFrame(out)[['ImageId', 'EncodedPixels']]
submission.to_csv('submission.csv', index=False)

Epoch 1:   1%|          | 384/40432 [00:30<24:41, 27.04it/s, loss=5.7994]